# ------------- User's settings -------------
This is the only section you need to make inputs

In [10]:
# Output from Step 3 is needed as the input here

input_directory = '../Data/Step3/Output/'

# ------------- Executable -------------
Execute the rest of the script

In [1]:
import os,cv2
import glob
import numpy 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

In [11]:
feature_vector = numpy.load(os.path.join(input_directory,'selected_features.npy'))

if len(feature_vector.shape) > 2:
    a = numpy.mean(feature_vector, axis = 1)
    del(feature_vector)
    feature_vector = numpy.mean(a, axis = 1)  


feature_vector.shape

(33437, 700)

In [12]:
# the shape of feature vectors should be (num_samples,length_of_each_feature) . eg (400,4096)

print ("feature_vector_shape:",feature_vector.shape) 
print ("num of images:",feature_vector.shape[0])
print ("size of individual feature vector:",feature_vector.shape[1])

# Load the features in a tensorflow variable 
features = tf.Variable(feature_vector, name='features')

feature_vectors_shape: (33437, 700)
num of images: 33437
size of individual feature vector: 700
Instructions for updating:
Colocations handled automatically by placer.


In [13]:
LOG_DIR = '.'
with tf.Session() as sess:
    saver = tf.train.Saver([features])

    sess.run(features.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = features.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = os.path.join(LOG_DIR, 'metadata_label.tsv')
    # Comment out if you don't want sprites
    embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite.png')
    embedding.sprite.single_image_dim.extend([48, 48])
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)